In [1]:
import glob
import os
import sys
import numpy as np
from scipy import stats
from scipy import io as sio
import scipy.spatial.distance as sp_distance
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
from scipy.stats import norm, zscore, pearsonr
from scipy.signal import gaussian, convolve
from sklearn import decomposition
import pandas as pd
import pickle


from brainiak.isc import isc
from brainiak.fcma.util import compute_correlation
import brainiak.funcalign.srm
from brainiak import image, io
from brainiak.isc import isc
import brainiak.funcalign.srm
from brainiak import image, io
from brainiak.eventseg.event import EventSegment
import matplotlib.patches as patches

In [2]:
srm_data_path = '/Volumes/ARCHIVES/thesis_pipeline/SRM_data'
roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']
figures_dir = '/Volumes/ARCHIVES/thesis_pipeline/figures/group_SRM_ROIs'
save_data_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/group_SRM_data'
os.makedirs(figures_dir,  exist_ok=True)
os.makedirs(save_data_dir, exist_ok=True)

roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']
group_srm_data = {}

# Load and prepare SRM data for each ROI
for roi in roi_names:
    srm_file_path = os.path.join(srm_data_path, f"{roi}_shareddata.pkl")
    
    if os.path.exists(srm_file_path):
        with open(srm_file_path, 'rb') as file:
            srm_data = pickle.load(file)  # Loaded data shape: [subjects][TRs x features]
        
        # Stack data across subjects and average across that axis to create group-level data
        group_data = np.mean(np.stack(srm_data, axis=0), axis=0)  # Resulting shape: [TRs x features]
        group_srm_data[roi] = group_data

        # Save the group-level data as a .npy file
        save_path = os.path.join(save_data_dir, f'{roi}_group_SRM_data.npy')
        np.save(save_path, group_data)

        print(f"Loaded, prepared, and saved group-level data for {roi} with shape {group_data.shape}")
    else:
        print(f"SRM data file for {roi} not found.")


Loaded, prepared, and saved group-level data for PTL with shape (90, 1291)
Loaded, prepared, and saved group-level data for ATL with shape (90, 1291)
Loaded, prepared, and saved group-level data for AG with shape (90, 1291)
Loaded, prepared, and saved group-level data for IFG with shape (90, 1291)
Loaded, prepared, and saved group-level data for MFG with shape (90, 1291)
Loaded, prepared, and saved group-level data for IFGorb with shape (90, 1291)


In [ ]:
# Path to the saved group-level data for IFG
ifg_data_path = '/Volumes/ARCHIVES/thesis_pipeline/data/group_SRM_data/IFG_group_SRM_data.npy'

# Load the IFG group-level SRM data
if os.path.exists(ifg_data_path):
    ifg_group_data = np.load(ifg_data_path)
    print(f"Loaded IFG group-level data with shape: {ifg_group_data.shape}")

    # Visualize the mean signal across all features for each TR
    mean_signal_per_TR = np.mean(ifg_group_data, axis=1)

    plt.figure(figsize=(12, 6))
    plt.plot(mean_signal_per_TR, color='blue', label='Mean Signal Across Features')
    plt.title('Mean Signal Across Features for Each TR - IFG')
    plt.xlabel('TRs')
    plt.ylabel('Mean Signal Value')
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("IFG group-level data file not found.")

In [4]:
group_srm_data_path = '/Volumes/ARCHIVES/thesis_pipeline/data/group_SRM_data'
save_hmm_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/group_level_HMM'

os.makedirs(save_hmm_dir, exist_ok=True)

n_states = 37  # Number of states for HMM segmentation

for roi in roi_names:
    group_data_path = os.path.join(group_srm_data_path, f'{roi}_group_SRM_data.npy')
    
    if os.path.exists(group_data_path):
        # Load group-level SRM data for the ROI
        group_data = np.load(group_data_path)
        print(f"Loaded group-level SRM data for {roi} with shape {group_data.shape}")

        # Check if data needs to be transposed
        if group_data.shape[0] < group_data.shape[1]:  # Check if features < TRs
            print("Transposing data to [features, TRs] for HMM fitting.")
            group_data = group_data.T

        # Initialize and fit HMM model
        n_states = 37  # Specify number of states
        hmm_model = EventSegment(n_states)
        print(f"Fitting HMM with {n_states} states for {roi}...")
        hmm_model.fit(group_data)  # Fit with correct shape

        # Save the HMM model and segmentation data
        hmm_save_path = os.path.join(save_hmm_dir, f'{roi}_HMM_model.pkl')
        with open(hmm_save_path, 'wb') as file:
            pickle.dump(hmm_model, file)

        # Extract event boundaries and save them
        event_bounds = np.where(np.diff(np.argmax(hmm_model.segments_[0], axis=1)))[0]
        event_bounds_save_path = os.path.join(save_hmm_dir, f'{roi}_event_bounds.npy')
        np.save(event_bounds_save_path, event_bounds)

        print(f"Saved HMM model and event boundaries for {roi}")
    else:
        print(f"Group-level data for {roi} not found.")

Loaded group-level SRM data for PTL with shape (90, 1291)
Transposing data to [features, TRs] for HMM fitting.
Fitting HMM with 37 states for PTL...
Saved HMM model and event boundaries for PTL
Loaded group-level SRM data for ATL with shape (90, 1291)
Transposing data to [features, TRs] for HMM fitting.
Fitting HMM with 37 states for ATL...
Saved HMM model and event boundaries for ATL
Loaded group-level SRM data for AG with shape (90, 1291)
Transposing data to [features, TRs] for HMM fitting.
Fitting HMM with 37 states for AG...
Saved HMM model and event boundaries for AG
Loaded group-level SRM data for IFG with shape (90, 1291)
Transposing data to [features, TRs] for HMM fitting.
Fitting HMM with 37 states for IFG...
Saved HMM model and event boundaries for IFG
Loaded group-level SRM data for MFG with shape (90, 1291)
Transposing data to [features, TRs] for HMM fitting.
Fitting HMM with 37 states for MFG...
Saved HMM model and event boundaries for MFG
Loaded group-level SRM data for I

In [6]:
# Path to the saved HMM event boundaries and model for visualization
hmm_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/group_level_HMM'
roi_to_visualize = 'IFG'  # Example ROI

# Load and visualize the HMM event boundaries
event_bounds_path = os.path.join(hmm_dir, f'{roi_to_visualize}_event_bounds.npy')
group_data_path = os.path.join('/Volumes/ARCHIVES/thesis_pipeline/data/group_SRM_data', f'{roi_to_visualize}_group_SRM_data.npy')

roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']

# Loop through each ROI to create and save the visualization
for roi in roi_names:
    event_bounds_path = os.path.join(hmm_dir, f'{roi}_event_bounds.npy')
    group_data_path = os.path.join(group_srm_data_path, f'{roi}_group_SRM_data.npy')
    
    if os.path.exists(event_bounds_path) and os.path.exists(group_data_path):
        event_bounds = np.load(event_bounds_path)
        group_data = np.load(group_data_path)
        
        # Visualize the event boundaries on a correlation matrix of the group data
        corr_matrix = np.corrcoef(group_data.T)  # Compute TR-by-TR correlation matrix
        
        plt.figure(figsize=(12, 10))
        plt.imshow(corr_matrix, cmap='viridis')
        plt.colorbar(label='Correlation')
        plt.title(f'Correlation Matrix with Event Boundaries - {roi}')
        plt.xlabel('TR')
        plt.ylabel('TR')
        
        # Plot the event boundaries as vertical and horizontal lines
        for bound in event_bounds:
            plt.axvline(x=bound, color='red', linestyle='--', linewidth=1)
            plt.axhline(y=bound, color='red', linestyle='--', linewidth=1)
        
        plt.tight_layout()
        
        # Save the figure to the figures directory
        figure_save_path = os.path.join(figures_dir, f'{roi}_HMM_correlation_matrix.png')
        plt.savefig(figure_save_path, dpi=300)
        plt.close()
        
        print(f"Saved visualization for {roi} at {figure_save_path}")
    else:
        print(f"Event boundaries or group data for {roi} not found.")

Saved visualization for PTL at /Volumes/ARCHIVES/thesis_pipeline/figures/group_SRM_ROIs/PTL_HMM_correlation_matrix.png
Saved visualization for ATL at /Volumes/ARCHIVES/thesis_pipeline/figures/group_SRM_ROIs/ATL_HMM_correlation_matrix.png
Saved visualization for AG at /Volumes/ARCHIVES/thesis_pipeline/figures/group_SRM_ROIs/AG_HMM_correlation_matrix.png
Saved visualization for IFG at /Volumes/ARCHIVES/thesis_pipeline/figures/group_SRM_ROIs/IFG_HMM_correlation_matrix.png
Saved visualization for MFG at /Volumes/ARCHIVES/thesis_pipeline/figures/group_SRM_ROIs/MFG_HMM_correlation_matrix.png
Saved visualization for IFGorb at /Volumes/ARCHIVES/thesis_pipeline/figures/group_SRM_ROIs/IFGorb_HMM_correlation_matrix.png


In [8]:
import seaborn as sns 

# Loop through each ROI to inspect and plot event boundaries
for roi in roi_names:
    event_bounds_path = os.path.join(hmm_dir, f'{roi}_event_bounds.npy')
    
    if os.path.exists(event_bounds_path):
        event_bounds = np.load(event_bounds_path)
        
        # Print summary information about event boundaries
        print(f"ROI: {roi}")
        print(f"  Number of Event Boundaries: {len(event_bounds)}")
        print(f"  Event Boundaries (first 10): {event_bounds[:10]}")
        print(f"  Max Boundary: {np.max(event_bounds)}, Min Boundary: {np.min(event_bounds)}")
        
        # Create a prettier histogram for event boundaries distribution
        plt.figure(figsize=(10, 6))
        sns.histplot(event_bounds, bins=20, kde=True, color='skyblue', edgecolor='black')
        plt.title(f'{roi}', fontsize=16, weight='bold')
        plt.xlabel('TRs', fontsize=14, weight='bold')
        plt.ylabel('Frequency', fontsize=14, weight='bold')
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        
        # Save the plot
        plot_save_path = os.path.join(figures_dir, f'{roi}_event_boundaries_distribution.png')
        plt.tight_layout()
        plt.savefig(plot_save_path, dpi=300)
        plt.close()
        
        print(f"Saved event boundaries distribution plot for {roi} at {plot_save_path}")
        
    else:
        print(f"Event boundaries file for {roi} not found.")


ROI: PTL
  Number of Event Boundaries: 36
  Event Boundaries (first 10): [ 29  79 106 144 165 199 217 244 291 325]
  Max Boundary: 1261, Min Boundary: 29
Saved event boundaries distribution plot for PTL at /Volumes/ARCHIVES/thesis_pipeline/figures/group_SRM_ROIs/PTL_event_boundaries_distribution.png
ROI: ATL
  Number of Event Boundaries: 36
  Event Boundaries (first 10): [ 30  75 103 129 164 191 217 244 294 324]
  Max Boundary: 1254, Min Boundary: 30
Saved event boundaries distribution plot for ATL at /Volumes/ARCHIVES/thesis_pipeline/figures/group_SRM_ROIs/ATL_event_boundaries_distribution.png
ROI: AG
  Number of Event Boundaries: 36
  Event Boundaries (first 10): [ 22  42  77 107 164 196 218 245 284 321]
  Max Boundary: 1256, Min Boundary: 22
Saved event boundaries distribution plot for AG at /Volumes/ARCHIVES/thesis_pipeline/figures/group_SRM_ROIs/AG_event_boundaries_distribution.png
ROI: IFG
  Number of Event Boundaries: 36
  Event Boundaries (first 10): [ 23  43  81 108 164 192 21